In [ ]:
!pip install -q -U keras-tuner
from google.colab import drive
# drive.mount('/content/drive')

     |████████████████████████████████| 71kB 8.5MB/s 


In [ ]:
# Imports
import gdown
import random
import os
import json
import shutil
import csv   
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import scipy.misc
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow.keras.layers.experimental.preprocessing as keras_preproc
import kerastuner as kt
from pprint import pprint

In [ ]:
# Download provided dataset
!wget files.brainfriz.com/train_images.zip # secondary link for images
!unzip -qq -o train_images.zip
!rm train_images.zip
!gdown --id "1xbEVK_NigW_5ngwKMHvuOTehYhT2v2WF" # labels
!gdown --id "1SvI9dN2_25c2OlevwK4TjmzBNysjE_PO" # label mapping

--2021-03-24 06:42:38--  http://files.brainfriz.com/train_images.zip
Resolving files.brainfriz.com (files.brainfriz.com)... 138.201.201.196
Connecting to files.brainfriz.com (files.brainfriz.com)|138.201.201.196|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.brainfriz.com/train_images.zip [following]
--2021-03-24 06:42:38--  https://files.brainfriz.com/train_images.zip
Connecting to files.brainfriz.com (files.brainfriz.com)|138.201.201.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2569658627 (2.4G) [application/zip]
Saving to: ‘train_images.zip’

train_images.zip    100%[===================>]   2.39G  30.8MB/s    in 80s     

2021-03-24 06:43:58 (30.8 MB/s) - ‘train_images.zip’ saved [2569658627/2569658627]

Downloading...
From: https://drive.google.com/uc?id=1xbEVK_NigW_5ngwKMHvuOTehYhT2v2WF
To: /content/train.csv
100% 358k/358k [00:00<00:00, 5.72MB/s]
Downloading...
From: https://drive.google.com

In [ ]:
IMG_SIZE = 512

def augment_image(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
  img4d = tf.expand_dims(img, 0)
  data_augmentation = tf.keras.Sequential([
    keras_preproc.Resizing(IMG_SIZE, IMG_SIZE),
    keras_preproc.RandomRotation(0.2),
    keras_preproc.RandomZoom((0,-0.3)),
  ])

  aug_img_arr = data_augmentation(img4d)

  aug_img = Image.fromarray(aug_img_arr.numpy()[0].astype(np.uint8))
    
  return aug_img

def add_train_datapoint_cassava(image, image_id, label, train_images_dir_path, train_csv_path):
    datapoint = dict({
        'image_id': image_id,
        'label': label,
    })
    
    if not os.path.exists(train_images_dir_path):
        os.makedirs(train_images_dir_path)
      
    image.save(train_images_dir_path + str(image_id)) # save
  
    with open(train_csv_path, 'a') as f:
      writer = csv.DictWriter(f, ['image_id', 'label'])
      writer.writerow(datapoint)


In [ ]:
def get_data_with_label(data, label):
  return data.loc[data['label'] == label]

def create_seperate_set(set_data, dir_name):
  img_ids = set_data['image_id']
  for id in img_ids:
    os.rename(f"/content/train_images/{id}", f"/content/{dir_name}/{id}")

In [ ]:
!mkdir val_images
!mkdir test_images

random_state = np.random.RandomState(69420)

train_data = pd.read_csv('train.csv')

val_data = pd.DataFrame(data=None, columns=train_data.columns) 
test_data = pd.DataFrame(data=None, columns=train_data.columns) 

for i in range(5):
  test_rows = train_data[train_data["label"] == i].sample(n=200, random_state=random_state)
  test_data = test_data.append(test_rows)
  train_data = train_data.drop(test_rows.index)

create_seperate_set(test_data, 'test_images')

for i in range(5):
  val_rows = train_data[train_data["label"] == i].sample(n=200, random_state=random_state)
  val_data = val_data.append(val_rows)
  train_data = train_data.drop(val_rows.index)

create_seperate_set(val_data, 'val_images')


val_data.to_csv('val_data.csv')
test_data.to_csv('test_data.csv')


with open('augmented_data.csv', 'a') as f:
      writer = csv.DictWriter(f, ['image_id', 'label'])
      writer.writerow(dict({
        'image_id': 'image_id',
        'label': 'label',
      }))

unique_labels = set(test_data['label'])

n_aug_for_balance = {}

for label in unique_labels:
  n_aug_for_balance[label] = len(train_data) - len(get_data_with_label(train_data, label))

n_aug_for_balance_largest_class = min(n_aug_for_balance.values())

for label in unique_labels:
  n_aug_for_balance[label] -= n_aug_for_balance_largest_class

for label in n_aug_for_balance.keys():
  data_filtered_by_label = get_data_with_label(train_data, label)
  for i in range(n_aug_for_balance[label]):
    print(label, i)
    datapoint_to_augment = data_filtered_by_label.iloc[i % len(data_filtered_by_label)]

    image = cv2.imread('train_images/' + datapoint_to_augment['image_id'])
    augmented_image = augment_image(image)
    
    add_train_datapoint_cassava(
        image=augmented_image, 
        image_id='aug_'+str(label)+'_'+str(i)+'_'+datapoint_to_augment['image_id'], 
        label=datapoint_to_augment['label'],
        train_images_dir_path='augmented_images/',
        train_csv_path='augmented_data.csv'
    )

In [ ]:
for file_name in os.listdir('augmented_images/'):
    shutil.move(os.path.join('augmented_images', file_name), 'train_images')

augmented_data = pd.read_csv('augmented_data.csv')
pd.concat([train_data, augmented_data], ignore_index=True).to_csv('train_data.csv')

In [ ]:
!zip -r train_images.zip train_images
!zip -r val_images.zip val_images
!zip -r test_images.zip test_images

In [ ]:
!mv /content/train_images.zip /content/drive/MyDrive/train_images.zip
!mv /content/val_images.zip /content/drive/MyDrive/val_images.zip
!mv /content/test_images.zip /content/drive/MyDrive/test_images.zip
!mv /content/train_data.csv /content/drive/MyDrive/train_data.csv
!mv /content/val_data.csv /content/drive/MyDrive/val_data.csv
!mv /content/test_data.csv /content/drive/MyDrive/test_data.csv